In [4]:
! sudo rm -rf /usr/local/bin/ninja
! sudo wget -qO /usr/local/bin/ninja.gz https://github.com/ninja-build/ninja/releases/latest/download/ninja-linux.zip
! sudo gunzip /usr/local/bin/ninja.gz
! sudo chmod a+x /usr/local/bin/ninja
! ninja --version

1.11.1


In [3]:
!pip install timm ftfy pyspng imageio-ffmpeg pyopengl glfw imgui opencv-contrib-python

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.5/206.5 kB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 27.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 25.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 42.6 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 32.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 7.2 MB/s eta 0:00:00


In [5]:
! pip install --upgrade opencv-python
! pip install --upgrade opencv-contrib-python

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 MB 35.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.6.0.66
    Uninstalling opencv-python-4.6.0.66:
      Successfully uninstalled opencv-python-4.6.0.66
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [6]:
! python -m spacy download en_core_web_lg

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.2 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [16]:
# Copyright (c) 2021, NVIDIA CORPORATION & AFFILIATES.  All rights reserved.
#
# NVIDIA CORPORATION and its licensors retain all intellectual property
# and proprietary rights in and to this software, related documentation
# and any modifications thereto.  Any use, reproduction, disclosure or
# distribution of this software and related documentation without an express
# license agreement from NVIDIA CORPORATION is strictly prohibited.

"""Generate lerp videos using pretrained network pickle."""

import copy
import os
import re
from typing import List, Optional, Tuple, Union

import click
import dnnlib
import imageio
import numpy as np
import scipy.interpolate
import torch
from tqdm import tqdm

import legacy
from torch_utils import gen_utils

# import cv2
#from cv2 import dnn_superres

#----------------------------------------------------------------------------

def layout_grid(img, grid_w=None, grid_h=1, float_to_uint8=True, chw_to_hwc=True, to_numpy=True):
    batch_size, channels, img_h, img_w = img.shape
    if grid_w is None:
        grid_w = batch_size // grid_h
    assert batch_size == grid_w * grid_h
    if float_to_uint8:
        img = (img * 127.5 + 128).clamp(0, 255).to(torch.uint8)
    img = img.reshape(grid_h, grid_w, channels, img_h, img_w)
    img = img.permute(2, 0, 3, 1, 4)
    img = img.reshape(channels, grid_h * img_h, grid_w * img_w)
    if chw_to_hwc:
        img = img.permute(1, 2, 0)
    if to_numpy:
        img = img.cpu().numpy()
    return img

#----------------------------------------------------------------------------

def gen_interp_video(G, mp4: str, seeds, shuffle_seed=None, w_frames=60*4, kind='cubic', grid_dims=(1,1), num_keyframes=None, wraps=2, truncation_psi=1, device=torch.device('cuda'), centroids_path=None, class_idx=None, **video_kwargs):
    grid_w = grid_dims[0]
    grid_h = grid_dims[1]

    if num_keyframes is None:
        if len(seeds) % (grid_w*grid_h) != 0:
            raise ValueError('Number of input seeds must be divisible by grid W*H')
        num_keyframes = len(seeds) // (grid_w*grid_h)

    all_seeds = np.zeros(num_keyframes*grid_h*grid_w, dtype=np.int64)
    for idx in range(num_keyframes*grid_h*grid_w):
        all_seeds[idx] = seeds[idx % len(seeds)]

    if shuffle_seed is not None:
        rng = np.random.RandomState(seed=shuffle_seed)
        rng.shuffle(all_seeds)
    
    if class_idx is None:
        class_idx = [None] * len(seeds)
    elif len(class_idx) == 1:
        class_idx = [class_idx] * len(seeds)
    assert len(all_seeds) == len(class_idx), "Seeds and class-idx should have the same length"
    
    
    
    ws = []
    for seed, cls in zip(all_seeds, class_idx):
        ws.append(
            gen_utils.get_w_from_seed(G, 1, device, truncation_psi, seed=seed,
                                      centroids_path=centroids_path, class_idx=cls)
        )
        
    ws = torch.cat(ws)
    
    
    
    _ = G.synthesis(ws[:1]) # warm up
    
    ws = ws.reshape(grid_h, grid_w, num_keyframes, *ws.shape[1:])

    # Interpolation.
    grid = []
    for yi in range(grid_h):
        row = []
        for xi in range(grid_w):
            x = np.arange(-num_keyframes * wraps, num_keyframes * (wraps + 1))
            y = np.tile(ws[yi][xi].cpu().numpy(), [wraps * 2 + 1, 1, 1])
            interp = scipy.interpolate.interp1d(x, y, kind=kind, axis=0)
            row.append(interp)
        grid.append(row)
    
    
    # Render video.
    video_out = imageio.get_writer(mp4, mode='I', fps=30, codec='libx264', **video_kwargs)
    for frame_idx in tqdm(range(num_keyframes * w_frames)):
        imgs = []
        for yi in range(grid_h):
            for xi in range(grid_w):
                interp = grid[yi][xi]
                w = torch.from_numpy(interp(frame_idx / w_frames)).to(device)
                img = G.synthesis(ws=w.unsqueeze(0), noise_mode='const')[0]
                imgs.append(img)
        
        image_2d = layout_grid(torch.stack(imgs), grid_w=grid_w, grid_h=grid_h)
        
        video_out.append_data(image_2d)
    video_out.close()

#----------------------------------------------------------------------------


def parse_range(s: Union[str, List[int]]) -> List[int]:
    '''Parse a comma separated list of numbers or ranges and return a list of ints.

    Example: '1,2,5-10' returns [1, 2, 5, 6, 7]
    '''
    if isinstance(s, list): return s
    ranges = []
    range_re = re.compile(r'^(\d+)-(\d+)$')
    for p in s.split(','):
        m = range_re.match(p)
        if m:
            ranges.extend(range(int(m.group(1)), int(m.group(2))+1))
        else:
            ranges.append(int(p))
    return ranges

#----------------------------------------------------------------------------

def parse_tuple(s: Union[str, Tuple[int,int]]) -> Tuple[int, int]:
    '''Parse a 'M,N' or 'MxN' integer tuple.

    Example:
        '4x2' returns (4,2)
        '0,1' returns (0,1)
    '''
    if isinstance(s, tuple): return s
    m = re.match(r'^(\d+)[x,](\d+)$', s)
    if m:
        return (int(m.group(1)), int(m.group(2)))
    raise ValueError(f'cannot parse tuple {s}')

#----------------------------------------------------------------------------

In [20]:
model_dir = './imagenet64.pkl'


print('Loading networks from "%s"...' % model_dir)
device = torch.device('cuda')
with dnnlib.util.open_url(model_dir) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device) # type: ignore



Loading networks from "./imagenet64.pkl"...


In [9]:
import spacy
import json
import random
import math 
import time

nlp = spacy.load('en_core_web_lg')


labelTxt = [None] * 1000

# Using readlines()
file1 = open('imagenet_idx2labels.txt', 'r')
Lines = file1.readlines()
 
# Strips the newline character
for line in Lines:
    line = line[1:]
    split = line.replace(', ', ',').split(': ')
    
    class_idx = int(split[0])
    
    class_split = split[1][1: -3].split(',')
    
    #print(', '.join(class_split))
    
    # print(class_idx)
    
    labelTxt[class_idx] = class_split
    

In [19]:
outputLocation = 'lerp.mp4'

changeTime = 5
framerate = 30
amountPerClass = 1

def topicToSeed(topic):    
    start = time.time()
    
    closestIndex = 0
    sim = 0
    
    topicNLP = nlp(topic)
    
    for i in range(len(labelTxt)):
        
        if labelTxt == None:
            continue
                
        #for item in labelTxt[i]:
        
        item = labelTxt[i][0]
        item_nlp = nlp(item)
        cur_sim = topicNLP.similarity(item_nlp)
            
        if cur_sim > sim:
            sim = cur_sim
            closestIndex = i
    
    print(topic, closestIndex,":", time.time() - start)
    
    return closestIndex

def recieveInput(videoTopics):

    # good_idx = [506, 580, 599, 562, 611, 614, 624, 646, 672, 682, 706, 711, 750, 780, 789, 795, 802, 812, 815, 825, 832 , 835 , 839, 988, 985, 979, 967, 973, 975, 978, 980, 983]
    
    
    class_idx = []
    
    for topic in videoTopics:
        class_idx.append(topicToSeed(topic))
    
    for _ in range(6):
        class_idx.append(good_idx[random.randint(0, len(good_idx))])
    
    seeds = [random.randint(0,10000) for _ in range(len(class_idx))]
    
    num_keyframes = len(class_idx)
    
    w_frames = (framerate * changeTime)  
    
    print(f"Making a video {int(num_keyframes * w_frames / framerate)} seconds long.")
    
    gen_interp_video(G=G, mp4=outputLocation, bitrate='12M', grid_dims=(1,1), num_keyframes=num_keyframes, w_frames=w_frames, seeds=seeds, shuffle_seed=random.randint(0, 10000), truncation_psi=0.7, centroids_path=None, class_idx=class_idx)

    
recieveInput(['webs', 'rocket', 'snowboarding'])

Making a video 30 seconds long.


100%|██████████| 900/900 [03:40<00:00,  4.08it/s]
